In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from scipy.sparse import csr_matrix

# 1.  Tập dữ liệu Enron-Spam
## Tải tập dữ liệu

In [73]:
base_file_path = '.'
train_file_path = f'{base_file_path}/train.csv'
val_file_path = f'{base_file_path}/val.csv'

In [74]:
df_train = pd.read_csv(train_file_path, index_col = 0)
df_val = pd.read_csv(val_file_path, index_col = 0)

In [75]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27284 entries, 0 to 33715
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Message ID  27284 non-null  int64  
 1   Subject     27055 non-null  object 
 2   Message     26932 non-null  object 
 3   Spam/Ham    27284 non-null  object 
 4   split       27284 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 1.2+ MB


In [76]:
df_val.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3084 entries, 23 to 33692
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Message ID  3084 non-null   int64  
 1   Subject     3055 non-null   object 
 2   Message     3049 non-null   object 
 3   Spam/Ham    3084 non-null   object 
 4   split       3084 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 144.6+ KB


In [77]:
df_train.head()

,Message ID,Subject,Message,Spam/Ham,split
0,0,christmas tree farm pictures,NaN,ham,0.038415
1,1,"vastar resources , inc .","gary , production from the high island larger ...",ham,0.696509
2,2,calpine daily gas nomination,- calpine daily gas nomination 1 . doc,ham,0.587792
3,3,re : issue,fyi - see note below - already done .\nstella\...,ham,-0.055438
5,5,mcmullen gas for 11 / 99,"jackie ,\nsince the inlet to 3 river plant is ...",ham,-0.419658


In [78]:
df_val.head()

,Message ID,Subject,Message,Spam/Ham,split
23,23,miscellaneous,- - - - - - - - - - - - - - - - - - - - - - fo...,ham,-0.351998
24,24,re : purge of old contract _ event _ status,fyi - what do you all think ?\n- - - - - - - -...,ham,0.257704
32,32,valero 8018 and 1394,it is my understanding the outages valero incu...,ham,0.091200
37,37,01 / 00 natural gas nomination,enron methanol company nominates the following...,ham,-1.745133
43,43,re : misc . questions,- - - - - - - - - - - - - - - - - - - - - - fo...,ham,-1.911987


# 2. Tiền xử lý dữ liệu
# Sau khi khám phá dữ liệu:
- Các cột có giá trị bị thiếu (Nan) là: Subject, Message
- Không tồn tại các dòng lặp
- Nội dung Subject, Message bao gồm các ký tự chữ (in hoa, in thường), số, ký tự đặc biệt và các dấu câu
# Phương pháp tiền xử lý dữ liệu:
- Chuyển đổi nhãn 'ham'/'spam' thành số 0/1
- Xử lý giá trị thiếu (Nan) bằng chuỗi ký tự rỗng
- Chuẩn hóa văn bản, chuyển thành chữ thường
- Kết hợp Subject và Message thành một cột văn bản duy nhất

In [79]:
# Xử lý giá trị thiếu (Nan) bằng chuỗi ký tự rỗng
df_train['Subject'] = df_train['Subject'].fillna('')
df_train['Message'] = df_train['Message'].fillna('')

df_val['Subject'] = df_val['Subject'].fillna('')
df_val['Message'] = df_val['Message'].fillna('')

In [80]:
# Kết hợp Subject và Message thành một cột văn bản duy nhất
df_train['full_text'] = df_train['Subject'] + " " + df_train['Message']
df_val['full_text'] = df_val['Subject'] + " " + df_val['Message']

In [81]:
# Chuẩn hóa văn bản, chuyển thành chữ thường
def preprocess_text(text):
    text = text.lower()        
    return text

df_train['full_text'] = df_train['full_text'].apply(preprocess_text)
df_val['full_text'] = df_val['full_text'].apply(preprocess_text)

In [82]:
# Chuyển đổi nhãn 'ham'/'spam' thành số 0/1
label_mapping = {'ham': 0, 'spam': 1}
df_train['label'] = df_train['Spam/Ham'].map(label_mapping)
df_val['label'] = df_val['Spam/Ham'].map(label_mapping)

In [83]:
print('Size of df_train:',len(df_train))
df_train.head()

Size of df_train: 27284


,Message ID,Subject,Message,Spam/Ham,split,full_text,label
0,0,christmas tree farm pictures,,ham,0.038415,christmas tree farm pictures,0
1,1,"vastar resources , inc .","gary , production from the high island larger ...",ham,0.696509,"vastar resources , inc . gary , production fro...",0
2,2,calpine daily gas nomination,- calpine daily gas nomination 1 . doc,ham,0.587792,calpine daily gas nomination - calpine daily g...,0
3,3,re : issue,fyi - see note below - already done .\nstella\...,ham,-0.055438,re : issue fyi - see note below - already done...,0
5,5,mcmullen gas for 11 / 99,"jackie ,\nsince the inlet to 3 river plant is ...",ham,-0.419658,"mcmullen gas for 11 / 99 jackie ,\nsince the i...",0


In [84]:
print('Size of df_val:',len(df_val))
df_val.head()

Size of df_val: 3084


,Message ID,Subject,Message,Spam/Ham,split,full_text,label
23,23,miscellaneous,- - - - - - - - - - - - - - - - - - - - - - fo...,ham,-0.351998,miscellaneous - - - - - - - - - - - - - - - - ...,0
24,24,re : purge of old contract _ event _ status,fyi - what do you all think ?\n- - - - - - - -...,ham,0.257704,re : purge of old contract _ event _ status fy...,0
32,32,valero 8018 and 1394,it is my understanding the outages valero incu...,ham,0.091200,valero 8018 and 1394 it is my understanding th...,0
37,37,01 / 00 natural gas nomination,enron methanol company nominates the following...,ham,-1.745133,01 / 00 natural gas nomination enron methanol ...,0
43,43,re : misc . questions,- - - - - - - - - - - - - - - - - - - - - - fo...,ham,-1.911987,re : misc . questions - - - - - - - - - - - - ...,0


In [85]:
X_train = df_train['full_text']
y_train = df_train['label']

X_val = df_val['full_text']
y_val = df_val['label']

## 3. Model Naive Bayes Classifier

In [86]:
class MyNaiveBayesClassifier:
    def __init__(self, alpha=1.0):
        """
        Khởi tạo classifier.
        alpha: Tham số làm mịn Laplace.
        log_class_priors_: Lưu trữ log của xác suất tiên nghiệm cho mỗi lớp.
        log_likelihoods_: Lưu trữ log của xác suất khả năng cho mọi từ trong từ điển đối với mỗi lớp.
        vocabulary_: Lưu trữ tập hợp tất cả các từ duy nhất xuất hiện trong toàn bộ tập dữ liệu huấn luyện.
        word_to_index_: Lưu trữ các chỉ số của các từ trong từ điển.
        vocab_size_: Kích thước của từ điển.
        classes_: Lưu trữ danh sách các nhãn lớp duy nhất mà mô hình đã học được từ tập huấn luyện.
        """
        self.alpha = alpha
        self.log_class_priors_ = {}
        self.log_likelihoods_ = {}
        self.vocabulary_ = []
        self.word_to_index_ = {}
        self.vocab_size_ = 0
        self.classes_ = np.array([])

    def _build_vocabulary(self, X_texts, min_df = 1, max_df = 1.0):
        """ Xây dựng từ điển từ danh sách các văn bản. """
        word_doc_counts = Counter()
        all_unique_words_in_docs = []

        for text in X_texts:
            words_in_doc = set(text.split()) # Set các từ duy nhất trong tài liệu này
            for word in words_in_doc:
                word_doc_counts[word] += 1
            all_unique_words_in_docs.append(words_in_doc) # Không cần thiết nếu chỉ xây dựng vocab

        n_docs = len(X_texts)

        actual_min_df = min_df if isinstance(min_df, int) else int(min_df * n_docs)
        actual_max_df = max_df if isinstance(max_df, int) else int(max_df * n_docs)

        filtered_vocabulary = []
        for word, count in word_doc_counts.items():
            if actual_min_df <= count <= actual_max_df:
                filtered_vocabulary.append(word)

        self.vocabulary_ = sorted(filtered_vocabulary)
        self.word_to_index_ = {word: i for i, word in enumerate(self.vocabulary_)}
        self.vocab_size_ = len(self.vocabulary_)

        if self.vocab_size_ == 0:
            print("Warning: Từ điển rỗng.")

    def _texts_to_sparse_bow_vectors(self, X_texts):
        """ Chuyển danh sách văn bản thành ma trận CSR Bag-of-Words thưa. """
        if self.vocab_size_ == 0:
            print("Warning: Từ điển rỗng, không thể tạo vector BoW.")
            return csr_matrix((len(X_texts), 0), dtype=int)

        data = []
        row_indices = []
        col_indices = []

        for i, text in enumerate(X_texts):
            word_counts_in_doc = Counter(text.split())
            for word, count in word_counts_in_doc.items():
                if word in self.word_to_index_:
                    j = self.word_to_index_[word]
                    data.append(count)
                    row_indices.append(i)
                    col_indices.append(j)

        sparse_bow_vectors = csr_matrix((data, (row_indices, col_indices)),
                                        shape=(len(X_texts), self.vocab_size_),
                                        dtype=int)
        return sparse_bow_vectors
    
    def fit(self, X_train_text, y_train_labels, min_df=5, max_df=0.95):
        if hasattr(X_train_text, 'tolist'): X_train_text = X_train_text.tolist()
        if hasattr(y_train_labels, 'tolist'): y_train_labels = y_train_labels.tolist()

        n_samples = len(X_train_text)
        if n_samples == 0:
            print("Warning: Tập huấn luyện rỗng.")
            return
        if len(X_train_text) != len(y_train_labels):
            print("Error: Độ dài X_train_text và y_train_labels không khớp.")
            return

        self.classes_ = np.unique(y_train_labels)

        # 1. Xây dựng từ điển
        self._build_vocabulary(X_train_text, min_df=min_df, max_df=max_df)
        if self.vocab_size_ == 0:
            print("Error: Từ điển rỗng.")
            return
        print(f"Từ điển được xây dựng với {self.vocab_size_} từ.")

        # 2. Vector hóa ma trận BoW thưa
        X_train_bow_sparse = self._texts_to_sparse_bow_vectors(X_train_text)

        # 3. Tính xác suất tiên nghiệm P(Lớp)
        for c in self.classes_:
            count_c = sum(1 for label in y_train_labels if label == c)
            self.log_class_priors_[c] = np.log(count_c / n_samples if n_samples > 0 else 1e-9)
            self.log_likelihoods_[c] = {}

        # 4. Tính word_counts_per_class và total_words_per_class từ ma trận BoW thưa
        word_counts_per_class = {c: np.zeros(self.vocab_size_, dtype=np.float64) for c in self.classes_}
        total_words_per_class = {c: 0.0 for c in self.classes_}

        for i in range(n_samples):
            label = y_train_labels[i]
            doc_vector_sparse = X_train_bow_sparse.getrow(i)
            for word_idx, count in zip(doc_vector_sparse.indices, doc_vector_sparse.data):
                word_counts_per_class[label][word_idx] += count
                total_words_per_class[label] += count

        # 5. Tính xác suất khả năng P(từ | Lớp) với Laplace smoothing
        for c in self.classes_:
            current_total_words_in_class = total_words_per_class.get(c, 0.0)
            denominator = current_total_words_in_class + self.alpha * self.vocab_size_

            if denominator == 0:
                denominator = 1e-9
                print(f"Warning: Denominator bằng 0 cho lớp {c} khi tính likelihood.")

            for word_idx in range(self.vocab_size_):
                count_word_in_class = word_counts_per_class[c][word_idx]
                numerator = count_word_in_class + self.alpha
                self.log_likelihoods_[c][word_idx] = np.log(numerator / denominator)

            self.log_likelihoods_[c]['<UNK_token_log_likelihood>'] = np.log(self.alpha / denominator)


    def _predict_log_proba_single_bow(self, bow_vector_sparse_row):
        """ Dự đoán log xác suất cho một vector BoW thưa. """
        log_probas = {}
        if not self.classes_.size > 0 or self.vocab_size_ == 0:
            return log_probas

        for c in self.classes_:
            log_probas[c] = self.log_class_priors_.get(c, np.log(1e-9))
            for word_idx, count in zip(bow_vector_sparse_row.indices, bow_vector_sparse_row.data):
                term_log_likelihood = self.log_likelihoods_.get(c, {}).get(word_idx,
                                                                        self.log_likelihoods_.get(c,{}).get('<UNK_token_log_likelihood>', np.log(1e-9)))
                log_probas[c] += term_log_likelihood * count
        return log_probas

    def predict(self, X_test_text):
        predictions = []
        if self.classes_.size == 0 or self.vocab_size_ == 0:
            print("Error: Model chưa được huấn luyện hoặc từ điển rỗng.")
            default_pred = self.classes_[0] if self.classes_.size > 0 else 0
            return [default_pred] * len(X_test_text)

        if hasattr(X_test_text, 'tolist'): X_test_text = X_test_text.tolist()

        X_test_bow_sparse = self._texts_to_sparse_bow_vectors(X_test_text)

        if X_test_bow_sparse.shape[1] == 0 and self.vocab_size_ > 0:
             print("Warning: Không có từ nào trong X_test_text thuộc từ điển đã học. Dự đoán mặc định.")
             default_pred = self.classes_[0] if self.classes_.size > 0 else 0
             return [default_pred] * len(X_test_text)

        for i in range(X_test_bow_sparse.shape[0]):
            bow_vector_sparse_row = X_test_bow_sparse.getrow(i)
            log_probas = self._predict_log_proba_single_bow(bow_vector_sparse_row)
            
            if not log_probas:
                predictions.append(self.classes_[0] if self.classes_.size > 0 else 0)
                continue
            
            best_class = max(log_probas, key=log_probas.get)
            predictions.append(best_class)
            
        return np.array(predictions)

# Đánh giá thục tế

In [87]:
def calculate_confusion_matrix_elements(y_true, y_pred, pos_label = 1, neg_label = 0):
    """
    Tính toán TP, TN, FP, FN từ nhãn thật và nhãn dự đoán.
    pos_label: nhãn của lớp positive (spam: 1)
    neg_label: nhãn của lớp negative (ham: 0)
    """
    tp = 0  
    tn = 0  
    fp = 0 
    fn = 0

    y_true_list = np.array(y_true)
    y_pred_list = np.array(y_pred)

    for i in range(len(y_true)):
        true_val = y_true_list[i]
        pred_val = y_pred_list[i]

        if true_val == pos_label and pred_val == pos_label:
            tp += 1
        elif true_val == neg_label and pred_val == neg_label:
            tn += 1
        elif true_val == neg_label and pred_val == pos_label:
            fp += 1
        elif true_val == pos_label and pred_val == neg_label: 
            fn += 1
    return tp, tn, fp, fn

def my_accuracy_score(tp, tn, fp, fn):
    total = tp + tn + fp + fn
    if total == 0:
        return 0.0
    return (tp + tn) / total

def my_precision_score(tp, fp):
    if (tp + fp) == 0:
        return 0.0
    return tp / (tp + fp)

def my_recall_score(tp, fn):
    if (tp + fn) == 0:
        return 0.0
    return tp / (tp + fn)

def my_f1_score(precision, recall):
    if (precision + recall) == 0:
        return 0.0
    return 2 * (precision * recall) / (precision + recall)

In [88]:
def evaluate_model(y_true, y_pred, dataset_name="Dataset", pos_label=1, neg_label=0, class_labels=None):
    if len(y_true) == 0 or len(y_pred) == 0 or len(y_true) != len(y_pred):
        print(f"Error: Đầu vào {dataset_name} không hợp lệ len(y_true) = {len(y_true)}, len(y_pred) = {len(y_pred)}")
        return

    if class_labels is None:
        unique_labels = sorted(list(set(y_true) | set(y_pred)))
        if len(unique_labels) == 1:
             class_labels = [unique_labels[0], 1 - unique_labels[0]]
        else:
             class_labels = unique_labels[:2]

    if pos_label not in class_labels or neg_label not in class_labels:
        print(f"Warning: pos_label ({pos_label}) hoặc neg_label ({neg_label}) không có trong class_labels ({class_labels})")
        if len(class_labels) >=2:
            neg_label = class_labels[0]
            pos_label = class_labels[1]
        else: 
            # Chỉ có 1 lớp thì không thể tính precision/recall cho 2 lớp
            print(f"Chỉ có một lớp ({class_labels[0]}). Không thể tính precision/recall cho 2 lớp.")

            # Tính accuracy cho model
            correct_predictions = sum(1 for yt, yp in zip(y_true, y_pred) if yt == yp)
            accuracy_one_class = correct_predictions / len(y_true) if len(y_true) > 0 else 0
            print(f"--- Kết quả đánh giá trên {dataset_name} ---")
            print(f"Accuracy (cho lớp {class_labels[0]}): {accuracy_one_class:.4f}")

            # Confusion Matrix 1x2
            cm_one_class = np.array([[correct_predictions]])
            print("\nMa trận nhầm lẫn (1x1):")
            print(cm_one_class)
            return


    # Tính accuracy cho model
    tp_pos, tn_pos, fp_pos, fn_pos = calculate_confusion_matrix_elements(y_true, y_pred, pos_label=pos_label, neg_label=neg_label)
    accuracy = my_accuracy_score(tp_pos, tn_pos, fp_pos, fn_pos)

    # Metrics cho lớp Positive (Spam)
    precision_pos = my_precision_score(tp_pos, fp_pos)
    recall_pos = my_recall_score(tp_pos, fn_pos)
    f1_pos = my_f1_score(precision_pos, recall_pos)

    # Metrics cho lớp Negative (Ham)
    tp_neg = tn_pos
    fp_neg = fn_pos
    fn_neg = fp_pos

    precision_neg = my_precision_score(tp_neg, fp_neg)
    recall_neg = my_recall_score(tp_neg, fn_neg)
    f1_neg = my_f1_score(precision_neg, recall_neg)

    print(f"--- Kết quả đánh giá trên {dataset_name} ---")
    print(f"Accuracy: {accuracy:.4f}")

    print(f"\nMetrics cho lớp Positive ({pos_label}):")
    print(f"  Precision: {precision_pos:.4f}")
    print(f"  Recall: {recall_pos:.4f}")
    print(f"  F1-Score: {f1_pos:.4f}")

    print(f"\nMetrics cho lớp Negative ({neg_label}):")
    print(f"  Precision: {precision_neg:.4f}")
    print(f"  Recall: {recall_neg:.4f}")
    print(f"  F1-Score: {f1_neg:.4f}")

    # Confusion Matrix 2x2
    print("\nMa trận nhầm lẫn:")
    cm_display_order = class_labels
    cm_data = np.zeros((2,2), dtype=int)

    # Hàng 0: y_true là class_labels[0]
    # Hàng 1: y_true là class_labels[1]
    # Cột 0: y_pred là class_labels[0]
    # Cột 1: y_pred là class_labels[1]

    idx_neg = class_labels.index(neg_label)
    idx_pos = class_labels.index(pos_label)

    cm_data[idx_neg, idx_neg] = tn_pos
    cm_data[idx_neg, idx_pos] = fp_pos
    cm_data[idx_pos, idx_neg] = fn_pos
    cm_data[idx_pos, idx_pos] = tp_pos
    print(cm_data)

In [89]:
model = MyNaiveBayesClassifier(alpha=1.0)
model.fit(X_train, y_train)

Từ điển được xây dựng với 37830 từ.


In [90]:
y_train_pred = model.predict(X_train)
evaluate_model(y_train, y_train_pred, dataset_name="Train Set",
                pos_label=1, neg_label=0,
                class_labels=sorted(list(model.classes_)) if model.classes_.size > 0 else [0,1])

--- Kết quả đánh giá trên Train Set ---
Accuracy: 0.9859

Metrics cho lớp Positive (1):
  Precision: 0.9838
  Recall: 0.9884
  F1-Score: 0.9861

Metrics cho lớp Negative (0):
  Precision: 0.9880
  Recall: 0.9832
  F1-Score: 0.9856

Ma trận nhầm lẫn:
[[13201   225]
 [  161 13697]]


In [91]:
y_val_pred = model.predict(X_val)
evaluate_model(y_val, y_val_pred, dataset_name="Validation Set",
                pos_label=1, neg_label=0,
                class_labels=sorted(list(model.classes_)) if model.classes_.size > 0 else [0,1])

--- Kết quả đánh giá trên Validation Set ---
Accuracy: 0.9841

Metrics cho lớp Positive (1):
  Precision: 0.9834
  Recall: 0.9853
  F1-Score: 0.9843

Metrics cho lớp Negative (0):
  Precision: 0.9848
  Recall: 0.9829
  F1-Score: 0.9839

Ma trận nhầm lẫn:
[[1495   26]
 [  23 1540]]


# 4. Thử nghiệm thực tế
## Chức năng 1: Nhập và dự đoán một email bất ỳ

In [94]:
def predict_single_email(classifier, preprocessor, subject=None, message=None):
    """
    Cho phép người dùng nhập tiêu đề và nội dung email, tiền xử lý và dự đoán email đó là spam/ham.
    classifier: Mô hình phân loại email spam/ham
    preprocessor: Hàm tiền xử lý
    """
    if len(classifier.classes_) <= 0:
        print("Error: Mô hình chưa được huấn luyện")
        return

    if subject is None:
        subject = input("Nhập tiêu đề email: ")
    if message is None:
        message = input("Nhập nội dung email: ")

    full_email_text = subject + " " + message
    processed_email = preprocessor(full_email_text)

    print(f"\nNội dung đã xử lý: '{processed_email}'")

    email_bow_matrix_sparse = classifier._texts_to_sparse_bow_vectors([processed_email])
    if email_bow_matrix_sparse.shape[0] == 0 or email_bow_matrix_sparse.shape[1] == 0 :
        print("Không thể vector hóa email.")
        if classifier.log_class_priors_:
            default_prediction_label = max(classifier.log_class_priors_, key=classifier.log_class_priors_.get)
            class_name = "Spam" if default_prediction_label == 1 else "Ham"
            print(f"\n>>> Dự đoán mặc định: Email này là {class_name.upper()} ({default_prediction_label}).")
        else:
            print("\n>>> Không thể đưa ra dự đoán mặc định do thiếu thông tin prior.")
        return

    single_email_bow_vector_sparse_row = email_bow_matrix_sparse.getrow(0)
    prediction_proba_log = classifier._predict_log_proba_single_bow(single_email_bow_vector_sparse_row)

    if prediction_proba_log:
        prediction_label = max(prediction_proba_log, key=prediction_proba_log.get)
    else:
        print("Không thể tính toán log probabilities.")
        prediction_label = classifier.classes_[0] if classifier.classes_.size > 0 else 0

    print("Log probabilities cho các lớp:")
    if prediction_proba_log:
        for class_val, log_prob in prediction_proba_log.items():
            class_name = "Spam" if class_val == 1 else "Ham"
            print(f"  Lớp {class_name} ({class_val}): {log_prob:.4f}")
    else:
        print("  Không thể tính toán log probabilities (có thể do lỗi trong _predict_log_proba_single_bow).")

    if prediction_label == 1:
        print("\n>>> Dự đoán: Email này là SPAM.")
    elif prediction_label == 0:
        print("\n>>> Dự đoán: Email này là HAM.")
    else:
        print(f"\n>>> Dự đoán: Nhãn không xác định ({prediction_label}).")

In [95]:
# Ví dụ 1: SPAM
print("--- Thử nghiệm với email SPAM ---")
spam_subject = "VIAGRA Special Offer, FREE trial, limited time only"
spam_message = "Click here to claim your prize. Buy now cheap viagra pills. Winner confirmation."
predict_single_email(model, preprocess_text, spam_subject, spam_message)

--- Thử nghiệm với email SPAM ---

Nội dung đã xử lý: 'viagra special offer, free trial, limited time only click here to claim your prize. buy now cheap viagra pills. winner confirmation.'
Log probabilities cho các lớp:
  Lớp Ham (0): -147.2428
  Lớp Spam (1): -115.2543

>>> Dự đoán: Email này là SPAM.


In [96]:
# Ví dụ 2: HAM
print("--- Thử nghiệm với email HAM ---")
ham_subject = "Re: Project Update"
ham_message = "Hi team, please see attached the weekly report. Let me know if you have any questions. Thanks."
predict_single_email(model, preprocess_text, ham_subject, ham_message)

--- Thử nghiệm với email HAM ---

Nội dung đã xử lý: 're: project update hi team, please see attached the weekly report. let me know if you have any questions. thanks.'
Log probabilities cho các lớp:
  Lớp Ham (0): -99.9530
  Lớp Spam (1): -108.6254

>>> Dự đoán: Email này là HAM.


## Chức năng 2: Đọc và dự đoán file CSV

In [97]:
def evaluate_csv_file(classifier, preprocessor, evaluation_function, file_path = None):
    """
    Cho phép người dùng nhập tên file CSV, đọc file, tiền xử lý, dự đoán email đó là spam/ham và đánh giá kết quả.
    classifier: Mô hình phân loại email
    preprocessor: Hàm tiền xử lý
    evaluation_function: Hàm đánh giá kết quả
    """
    if len(classifier.classes_) == 0:
        print("Error: Mô hình chưa được huấn luyện")
        return

    if file_path is None:
        file_path = input("Nhập đường dẫn đến file CSV: ")

    try:
        df_test = pd.read_csv(file_path)
        print(f"'{file_path}' đọc thành công, có {len(df_test)} dòng.")

        required_columns = ['Subject', 'Message', 'Spam/Ham']
        if not all(col in df_test.columns for col in required_columns):
            print(f"Lỗi: File CSV phải chứa các cột: {', '.join(required_columns)}")
            return

        # Tiền xử lý dữ liệu
        df_test['Subject'] = df_test['Subject'].fillna('')
        df_test['Message'] = df_test['Message'].fillna('')
        df_test['full_text'] = df_test['Subject'] + " " + df_test['Message']
        df_test['full_text'] = df_test['full_text'].apply(preprocessor)

        # Đã định nghĩa phía trên: label_mapping = {'ham': 0, 'spam': 1}
        label_mapping_for_eval = label_mapping
        unknown_labels = df_test[~df_test['Spam/Ham'].isin(label_mapping_for_eval.keys())]['Spam/Ham'].unique()
        if len(unknown_labels) > 0:
            print(f"Warning: Các nhãn không xác định trong cột 'Spam/Ham': {unknown_labels}")
        df_test['true_label'] = df_test['Spam/Ham'].map(label_mapping_for_eval)
        
        # Loại bỏ các dòng có nhãn không xác định: true_label = Nan
        original_len = len(df_test)
        df_test.dropna(subset=['true_label'], inplace=True)
        if len(df_test) < original_len:
            print(f"Đã loại bỏ {original_len - len(df_test)} dòng với các nhãn không xác định.")

        if df_test.empty:
            print("Không còn dữ liệu để đánh giá sau khi xử lý nhãn.")
            return
        df_test['true_label'] = df_test['true_label'].astype(int)
        
        # Dự đoán kết quả bằng model
        X = df_test['full_text']
        y_true = df_test['true_label']
        y_pred = classifier.predict(X)

        # Đánh giá kết quả
        print(f"\nĐánh giá kết quả file '{file_path}'...")
        if hasattr(evaluation_function, '__call__'):
            if 'evaluate_model' in evaluation_function.__name__:
                evaluation_function(y_true, y_pred, f"CSV File: {file_path}",
                                     pos_label=1, neg_label=0,
                                     class_labels=sorted(list(classifier.classes_)) if classifier.classes_.size > 0 else [0,1])
            else:
                 print('Không tìm thấy hàm đánh giá.')
        else:
            print("Hàm đánh giá không hợp lệ.")


    except FileNotFoundError:
        print(f"Error: Không tìm thấy file '{file_path}'.")
    except pd.errors.EmptyDataError:
        print(f"Error: File '{file_path}' rỗng.")
    except Exception as e:
        print(f"Đã xảy ra lỗi không mong muốn khi xử lý file CSV: {e}")

In [98]:
test_file_path = f'{base_file_path}/val.csv'
evaluate_csv_file(model, preprocess_text, evaluate_model, test_file_path)

'./val.csv' đọc thành công, có 3084 dòng.

Đánh giá kết quả file './val.csv'...
--- Kết quả đánh giá trên CSV File: ./val.csv ---
Accuracy: 0.9841

Metrics cho lớp Positive (1):
  Precision: 0.9834
  Recall: 0.9853
  F1-Score: 0.9843

Metrics cho lớp Negative (0):
  Precision: 0.9848
  Recall: 0.9829
  F1-Score: 0.9839

Ma trận nhầm lẫn:
[[1495   26]
 [  23 1540]]
